In [1]:
import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 42.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.
   ━━━━━━━━

In [3]:
!pip install --no-cache-dir git+https://github.com/Shannu3766/Cosine-similarity.git
# @main#76574c3690a5ecf452ca0e2bfb25b34b38b6bec5
# !pip uninstall adalora_bi_eq13 -y

  Cloning https://github.com/Shannu3766/Cosine-similarity.git to /tmp/pip-req-build-p8pxsrtw
  Running command git clone --filter=blob:none --quiet https://github.com/Shannu3766/Cosine-similarity.git /tmp/pip-req-build-p8pxsrtw
  Resolved https://github.com/Shannu3766/Cosine-similarity.git to commit 86b97d9741e02e448ac5379c7369c49d30816542
  Preparing metadata (setup.py) ... done
  Created wheel for adalora_bi_eq13: filename=adalora_bi_eq13-0.1.0-py3-none-any.whl size=7602 sha256=d4c9ba0f625d0288a422d57ebcde5a310eccc1ff89ee8e204689e24a449f813f
  Stored in directory: /tmp/pip-ephem-wheel-cache-agn9hrcl/wheels/76/bb/9e/d30494391cfeaff12e87f494af145e11c56eaec39eff965930
Successfully built adalora_bi_eq13


In [4]:
import warnings
import os
import logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")

In [5]:
import os
import torch
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from peft import get_peft_model, LoraConfig, TaskType
os.environ["WANDB_DISABLED"] = "true"

2025-12-13 13:16:29.278959: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765631789.432490      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765631789.476850      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [6]:
MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
TASK = "rte"
LR = 2e-4
NUM_EPOCHS = 3
BATCH_SIZE = 8 
MAX_LEN = 256  
rank= 8

In [7]:
import time
def load_dataset_with_retry(path, name=None, max_retries=None, wait=2):

    attempt = 0

    while True:
        try:
            if name is not None:
                ds = load_dataset(path, name)
            else:
                ds = load_dataset(path)

            print(f"Dataset loaded successfully after {attempt} attempts.")
            return ds

        except Exception as e:
            attempt += 1
            wait_time = wait * min(5, attempt)  

            print(f"[Attempt {attempt}] Failed to load dataset: {e}")
            print(f"Retrying in {wait_time} seconds...\n")

            if max_retries is not None and attempt >= max_retries:
                print("Max retries reached. Raising error.")
                raise e

            time.sleep(wait_time)

dataset = load_dataset_with_retry("glue", TASK)


Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset loaded successfully after 0 attempts.


In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3000
    })
})

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

In [10]:
from transformers import DataCollatorWithPadding

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

MAX_LEN = 256 


def clean_text(s):
    if s is None:
        return ""

    return " ".join(str(s).strip().split())


def preprocess_function(examples):
    premises = [clean_text(p) for p in examples["sentence1"]]
    hypotheses = [clean_text(h) for h in examples["sentence2"]]

    inputs = [
        (
            f"Premise: {p}\n"
            f"Hypothesis: {h}\n"
            f"Question: Does the premise entail the hypothesis?\n"
            f"Answer:"
        )
        for p, h in zip(premises, hypotheses)
    ]

    tokenized = tokenizer(
        inputs,
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
    )
    return tokenized


encoded_dataset = dataset.map(preprocess_function, batched=True)

encoded_dataset = encoded_dataset.remove_columns(["sentence1", "sentence2", "idx"])

if "label" in encoded_dataset["train"].column_names and "labels" not in encoded_dataset["train"].column_names:
    encoded_dataset = encoded_dataset.rename_column("label", "labels")

encoded_dataset.set_format("torch")

data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="longest",
)


Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [11]:
print("Model started Loading")

Model started Loading


In [12]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=2,
    device_map="auto",
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True
)

print("Model Loaded")
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False 

`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model Loaded


In [13]:
print("Model Loaded")
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False 

Model Loaded


In [14]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=rank,                    
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=[   
        "q_proj", "k_proj", "v_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj"
    ]
)

In [15]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainable params: 6,311,936 || all params: 1,040,828,416 || trainable%: 0.6064


In [16]:
from peft.tuners.lora import LoraLayer

def get_lora_module_names(peft_model):
    names = []
    for name, module in peft_model.named_modules():
        if isinstance(module, LoraLayer):
            names.append(name)
    return names

lora_names = get_lora_module_names(model)   
print(f"🔢 Number of LoRA modules: {len(lora_names)}\n")

num_lora_modules = len(lora_names)
TOTAL_RANK_BUDGET = rank*num_lora_modules  

avg_rank_per_module = TOTAL_RANK_BUDGET / num_lora_modules
print(
    f"💡 With TOTAL_RANK_BUDGET={TOTAL_RANK_BUDGET} over "
    f"{num_lora_modules} modules, avg rank ≈ {avg_rank_per_module:.2f}"
)


🔢 Number of LoRA modules: 154

💡 With TOTAL_RANK_BUDGET=1232 over 154 modules, avg rank ≈ 8.00


In [17]:
training_args = TrainingArguments(
    output_dir=f"{MODEL_ID}-RTE-v2",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    logging_steps=20,
    metric_for_best_model="accuracy",
    fp16=False,             
    bf16=True,      
    report_to="none",
)

In [18]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from adaptive_lora_gradient.callbacks import AdaptiveLoRACallback
import torch



if "label" in encoded_dataset["train"].column_names and "labels" not in encoded_dataset["train"].column_names:
    encoded_dataset = encoded_dataset.rename_column("label", "labels")

keep_cols = {"input_ids", "attention_mask", "labels"}

for split in encoded_dataset.keys():
    drop_cols = [c for c in encoded_dataset[split].column_names if c not in keep_cols]
    if drop_cols:
        encoded_dataset[split] = encoded_dataset[split].remove_columns(drop_cols)

encoded_dataset.set_format(type="torch")

data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="longest",
)

train_dataset = encoded_dataset["train"]
eval_dataset = encoded_dataset["validation"]

EVAL_BATCH_SIZE = getattr(training_args, "per_device_eval_batch_size", 8)

val_dataloader = DataLoader(
    eval_dataset,
    batch_size=EVAL_BATCH_SIZE,
    shuffle=False,
    collate_fn=data_collator,
    pin_memory=torch.cuda.is_available(),
    num_workers=2,
)


adaptive_callback = AdaptiveLoRACallback(
    total_rank=TOTAL_RANK_BUDGET,
    val_dataloader=val_dataloader,
    score_smoothing_beta=0.8,   
    update_interval=2,        
    warmup_epochs=0,
    cooldown_epochs=1,  
    min_rank=4,
    tau=0.7
)

print("✅ Adaptive LoRA callback ready.")

✅ Adaptive LoRA callback ready.


In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[adaptive_callback],
)

print("🚀 Starting Training...")
trainer.train()

🚀 Starting Training...

--- AdaptiveLoRA: Preparing ranks for Epoch 1 ---
Computing Gradient Importance scores...


Computing Importance:   0%|          | 0/35 [00:00<?, ?it/s]

📊 Applied Score Smoothing (beta=0.8)
Rank Updates (Epoch 1):
  - base_model.model.model.layers.0.self_attn.q_proj: r=8 → 6 (Score: 0.2318)
  - base_model.model.model.layers.0.self_attn.k_proj: r=8 → 7 (Score: 0.4298)
  - base_model.model.model.layers.0.self_attn.v_proj: r=8 → 6 (Score: 0.3252)
  - base_model.model.model.layers.0.self_attn.o_proj: r=8 → 11 (Score: 0.7046)
  - base_model.model.model.layers.0.mlp.gate_proj: r=8 (Unchanged, Score: 0.4908)
  - base_model.model.model.layers.0.mlp.up_proj: r=8 (Unchanged, Score: 0.4835)
  - base_model.model.model.layers.0.mlp.down_proj: r=8 (Unchanged, Score: 0.4839)
  - base_model.model.model.layers.1.self_attn.q_proj: r=8 (Unchanged, Score: 0.4493)
  - base_model.model.model.layers.1.self_attn.k_proj: r=8 → 14 (Score: 0.8628)
  - base_model.model.model.layers.1.self_attn.v_proj: r=8 → 9 (Score: 0.5836)
  - base_model.model.model.layers.1.self_attn.o_proj: r=8 → 9 (Score: 0.5527)
  - base_model.model.model.layers.1.mlp.gate_proj: r=8 (Unchan

Epoch,Training Loss,Validation Loss,Accuracy
1,0.576200,0.506595,0.779783
2,0.290600,0.544486,0.826715
3,0.128200,1.304977,0.826715


📄 Epoch 1: Rank allocations logged to ./logs/adaptive_lora_epoch_logs.csv


--- AdaptiveLoRA: Preparing ranks for Epoch 2 ---
⏩ Skipping update (Interval=2).
📄 Epoch 2: Rank allocations logged to ./logs/adaptive_lora_epoch_logs.csv


--- AdaptiveLoRA: Preparing ranks for Epoch 3 ---
❄️ Cooldown Period. Skipping.
📄 Epoch 3: Rank allocations logged to ./logs/adaptive_lora_epoch_logs.csv



TrainOutput(global_step=936, training_loss=0.38154686694471246, metrics={'train_runtime': 3251.859, 'train_samples_per_second': 2.297, 'train_steps_per_second': 0.288, 'total_flos': 8191304942650368.0, 'train_loss': 0.38154686694471246, 'epoch': 3.0})

In [20]:
print("📊 Final Evaluation:")
results = trainer.evaluate()
print(results)

📊 Final Evaluation:


{'eval_loss': 1.3049768209457397, 'eval_accuracy': 0.8267148014440433, 'eval_runtime': 49.5615, 'eval_samples_per_second': 5.589, 'eval_steps_per_second': 0.706, 'epoch': 3.0}
